In [47]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
import numpy as np

In [48]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

X,y= make_classification(n_samples=1000, n_classes=2, n_features=20, n_informative=10, n_redundant=5, random_state=42)

X = torch.tensor(X, dtype=torch.float32) 
y = torch.tensor(y, dtype=torch.float32) 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [49]:
class MyNeuralNet(nn.Module):
    def __init__(self, feature_size):
        super().__init__()
        self.linear=nn.Linear(feature_size,3)
        self.relu=nn.ReLU()
        self.linear2=nn.Linear(3,1)
        self.sigmoid=nn.Sigmoid()

    def forward(self,X):

        out=self.linear(X)
        out=self.relu(out)
        out=self.linear2(out)
        out=self.sigmoid(out)

        return out

In [50]:
class CustomDatasetLoader(Dataset):
    def __init__(self,features,labeles):
        self.features=features
        self.labels=labeles

    def __len__(self):
        return self.features.shape[0]
    
    def __getitem__(self, index):
        return self.features[index], self.labels[index]

In [51]:
dataset=CustomDatasetLoader(X,y)

In [52]:
dataloader=DataLoader(dataset,batch_size=2,shuffle=True)

In [53]:
for batch_features, batch_labels in dataloader:
    print(batch_features)
    print(batch_labels)

tensor([[ 5.3190e+00,  1.1727e+00, -1.4471e+00, -5.7695e-01, -2.7695e-03,
          1.3080e+00,  5.5464e-01, -4.0568e+00,  7.8890e-01, -1.1497e+00,
          7.6314e-01, -1.8292e+00,  6.1673e+00, -1.6239e+00, -9.3792e-01,
         -8.7993e-01,  1.2541e+00,  7.2529e-01, -4.6324e+00, -1.3319e+00],
        [-1.3852e+00,  2.5951e+00,  9.1379e-01,  1.9369e+00, -3.9624e-01,
          8.4355e-01,  3.0679e-01, -3.9841e+00, -4.5193e-01, -1.1597e+00,
         -1.0267e+00,  3.1119e-01,  6.0025e-01, -2.1331e+00,  3.3005e+00,
          1.1275e+00,  1.9174e+00, -1.1865e+00,  1.5054e+00,  8.2766e-01]])
tensor([0., 0.])
tensor([[-0.9359,  1.3154, -1.1059,  0.1751,  3.7321, -1.8643,  2.2447,  0.6271,
          2.2181, -4.0189,  1.1350, -2.0370,  3.5289, -0.4397, -2.1603,  0.5574,
         -0.0086,  2.9488, -0.7236, -0.8088],
        [-2.1698, -0.3407, -0.8123,  1.5198, -0.3318,  0.1045, -0.1447, -2.8405,
         -0.1740, -3.9399,  0.4972, -0.6757, -0.9726,  1.6720,  3.9488,  2.6215,
         -1.7052, 

In [54]:
dataloader=DataLoader(dataset,batch_size=32,shuffle=True)

In [55]:
epochs=20
lr=.001

model=MyNeuralNet(X_train.shape[1])
optimizer=torch.optim.Adam(model.parameters(),lr=lr)


loss_function=nn.BCELoss()

In [56]:


for epoch in range(1,epochs):
    
    for batch_data,batch_labels in dataloader:
        
        ypred=model(batch_data)

        loss=loss_function(ypred,batch_labels.view(-1,1))

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

    print(f"Epoch {epoch} Loss {loss.item()}")



Epoch 1 Loss 0.7005072236061096
Epoch 2 Loss 0.6872711181640625
Epoch 3 Loss 0.6481716632843018
Epoch 4 Loss 0.6113388538360596
Epoch 5 Loss 0.5461999773979187
Epoch 6 Loss 0.4751887321472168
Epoch 7 Loss 0.6965402364730835
Epoch 8 Loss 0.6018180847167969
Epoch 9 Loss 0.2848292291164398
Epoch 10 Loss 0.38131260871887207
Epoch 11 Loss 0.39880821108818054
Epoch 12 Loss 0.5422412753105164
Epoch 13 Loss 0.3254002630710602
Epoch 14 Loss 0.195225790143013
Epoch 15 Loss 0.32769545912742615
Epoch 16 Loss 0.22424209117889404
Epoch 17 Loss 0.30263909697532654
Epoch 18 Loss 0.18782341480255127
Epoch 19 Loss 0.3106011152267456


In [57]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# Create test DataLoader
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Put model in evaluation mode
model.eval()

# Track correct predictions
correct = 0
total = 0

# Disable gradient tracking for testing
with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X = batch_X.float()  # ensure correct dtype
        outputs = model(batch_X)

        # For binary classification
        predicted = (outputs > 0.5).float().view(-1)  # threshold & flatten
        correct += (predicted == batch_y).sum().item()
        total += batch_y.size(0)

# Calculate accuracy
accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")


Test Accuracy: 87.50%
